## Data transformation

### Remover duplicados

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

La función `drop_duplicates()` permite eliminar registros duplicados ya sea que el valor este duplicado en toda la fila o en alguna/s de las columnas.


In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

In [ ]:
data.drop_duplicates(['k1', 'k2'])

### Mapear y transformar los datos

A partir de un diccionario, se puede crear una nueva columna para un Dataframe donde las claves del mismo se vinculen con una de las series y los valores formen parte de la nueva columna. 

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami', 'honey ham',
                           'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
# La función map() puede recibir un diccionario y transforma el parámetro que recibe de la clave en el valor
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

In [ ]:
# Podemos hacer esto mismo más explícitamente con expresiones lambda.
data['food'].map(lambda x: meat_to_animal[x.lower()])

### Reemplazar valores

El método `data.replace()` ofrece varias formas de efectuar reemplazos sobre una serie de Pandas
    1- Un valor viejo por un valor nuevo
    2- Una lista de valores viejos por un valor nuevo
    3- Una lista de valores viejos por una lista de valores nuevos
    4- Un diccionario que mapee valores nuevos y viejos


In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

In [ ]:
data.replace([-999, -1000], [0, np.nan])

In [ ]:
data.replace({-999: np.nan, -1000: 0})

### Renombrar el índice de los ejes 

Recordemos que los objetos de tipo Index se comportan como las tuplas: son inmutable. Por esta razón no puedo modificar directamente uno de sus elementos como sí puedo hacer con los valores de los Dataframe y Series.

¿Cómo hacemos entonces para cambiar el nombre de una única columna?

Em método `rename()` acepta un diccionario para cada eje donde las keys son los valores viejos y los values son los valores nuevos.


In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])

In [ ]:
data.index.map(str.upper)

In [ ]:
data.index = data.index.map(str.upper)
data

In [ ]:
# La función title hace que las palabras se vean en mayúscula, seguida de minúsculas.

data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

In [ ]:
# Siempre devuelve una referencia al DataFrame, aunque no quiera utilizarla. Notar el nombre que se le asigna.

data.rename(index={'INDIANA': 'NEVADA'}, inplace=True)
data

### Discretizar y binarizar

El proceso de transformar una variable numérica en categórica se llama discretización. 

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [ ]:
# La función 'cut' devuelve el intervalo abierto al que pertenece cada entrada
# Defino el intervalo previamente

bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

In [ ]:
# El resultado de pd.cut devuelve un objeto de la clase Categorical.
type(cats)

In [ ]:
# 'codes' devuelve el indice del intervalo al que pertenece cada entrada
cats.codes

In [ ]:
pd.value_counts(cats)

In [ ]:
# El intervalo se puede definir semicerrado o semiabierto.

pd.cut(ages, [18, 26, 36, 61, 100], right=False)

In [ ]:
# Asigno nombres a los interavlos

group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
serie_ages = pd.cut(ages, bins, labels=group_names)

In [ ]:
serie_ages.value_counts()

### Detectar y filtrar outliers

La definición estándar de "outlier", indica que son outliers todos aquellos valores que se encuentran más de 3 desvíos estándar por encima o por debajo de su media. 

In [ ]:
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000, 4))
data.sample(5)

In [ ]:
# Detectamos para una serie, todos los valores que se encuentran a más de 3 desvíos estándar.
col = data[3]
col[np.abs(col) > 3 * np.std(col)]

In [ ]:
# Para ver los outliers en todo el Dataframe
data.apply(lambda col:col[np.abs(col) > 3 * np.std(col)])

### Variables Dummies

La utilización de variables dummies, también conocida como "one hot encoding", se puede interpretar como el proceso inverso de la discretización.
En este caso tomamos variables categóricas y las transformamos en variables numéricas que siguen una distribución binomial con probabilidad p, donde p es la cantidad de veces que aparece la categoría sobre el total de datos.

Pandas cuenta con el método pd.get_dummies() que recibe una Serie o una lista de Series y realiza el one hot encoding.

Recordemos que una variable con k categorías se puede representar con k-1 variables. 

Por eso un parámetro clave de pd.get_dummies es ```drop_first = True``` que genera k-1 categorías en lugar de k. 


In [ ]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
pd.get_dummies(df['key'])

In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

Ahora veamos un caso con un dataset de películas

In [ ]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('movies.csv', sep=';', header=None,
                        names=mnames, encoding='latin1')
movies[:10]

In [ ]:
possible_genres = [x.split('|') for x in movies.genres]
possible_genres[0:2]

In [ ]:
# Aplanamos la lista de películas
generos_p = [item for sublist in possible_genres for item in sublist]
generos_p

In [ ]:
import numpy as np
genres_unicos = pd.Series(generos_p)
genres_ordenados = np.sort(genres_unicos.unique())
genres_ordenados

In [ ]:
# Inicializamos en ceros todos los géneros.
dummies = pd.DataFrame(np.zeros((len(movies), genres_ordenados.size)), columns=genres_ordenados)
dummies

In [ ]:
# Recorremos la columna original de géneros y asignamos 1 con fancy indexing a los que corresponden.
for i, gen in enumerate(movies.genres):
    dummies.loc[i, gen.split('|')] = 1

In [ ]:
# Unimos el dataframe original con el que contiene las dummies
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

## Manipulación de strings

### String object methods

Veamos algunos métodos de manipulación de Strings

##### Split y Join

In [ ]:
val = 'a,b,  guido'
val.split(',')

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

In [ ]:
first, second, third = pieces
first + '::' + second + '::' + third

In [ ]:
'::'.join(pieces)

In [ ]:
'guido' in val

In [ ]:
# Devuelve la posición más baja donde encuentra el patrón solicitado.
val.index(',')

In [ ]:
# Devuelve -1 si no la encuentra
val.find(':')

In [ ]:
# Este método en cambio, devuelve un ValueError

val.index(':')

In [ ]:
val.count(',')

In [ ]:
val

In [ ]:
val.replace(',', '::')

In [ ]:
val.replace(',', ' ')

## Example: USDA Food Database

In [ ]:
import json
db = json.load(open('foods.json'))
len(db)
import pandas as pd

In [ ]:
# Recibimos datos nutricionales de 6636 productos.
db[0:1]

In [ ]:
# Cada producto es un diccionario. Veamos la información asociada a cada uno.
db[0].keys()

In [ ]:
# A su vez la propiedad nutrientes, es un objeto con otras propiedades.
db[0]['nutrients'][0]

In [ ]:
nutrients = pd.DataFrame(db[0]['nutrients'])
nutrients[:7]

In [ ]:
# Creo un dataframe seleccionando únicamente una parte de las propiedades originales.
info_keys = ['description', 'group', 'id', 'manufacturer']
info = pd.DataFrame(db, columns=info_keys)

In [ ]:
info[:5]

In [ ]:
info.sample(20)

In [ ]:
pd.value_counts(info.group)

In [ ]:
pd.value_counts(info.manufacturer)

In [ ]:
# Ahora queremos graficar la importancia de los distintos grupos
% matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt


plt.figure(figsize=(20,6))
graph = sns.countplot(x="group", data=info)
graph.set_xticklabels(info.group,rotation=90)
plt.show()


** Seleccionando la información **

La mejor forma de mostrar los conteos de categorías es con un gráfico de barras.
Pero ¿qué pasa si tengo demasiadas categorías?
Puedo elegir representar únicamente las primeras...

In [ ]:
top_manufacturers = info.manufacturer.dropna().value_counts()

top_manufacturers[0:10]

In [ ]:
plt.figure(figsize=(20,6))
sns.barplot(top_manufacturers[1:11].index, top_manufacturers[1:11].values)
plt.show()